# 📈 Stock Trading Analysis Tool (Updated)
## Analyze Your Buy & Sell Decisions Against Moving Averages

This notebook will help you:
1. Download 5 years of historical data for all your stocks
2. Calculate 50-DMA and 200-DMA
3. Analyze each buy transaction (conditions at entry)
4. Analyze each sell transaction (conditions at exit)
5. Determine if you made the right calls
6. Calculate opportunity cost/gain

## Step 1: Install Required Libraries

In [1]:
!pip install yfinance openpyxl -q
print("✓ Libraries installed successfully!")

✓ Libraries installed successfully!


## Step 2: Upload Your Tax P&L Excel File

Click on the file upload button below and select your tax P&L Excel file

In [2]:
from google.colab import files
import pandas as pd

print("=" * 80)
print("TAX P&L FILE UPLOAD")
print("=" * 80)

# Ask user how many files they want to upload
while True:
    try:
        num_files = int(input("\nHow many Tax P&L files do you want to upload? (Enter a number): "))
        if num_files > 0:
            break
        else:
            print("Please enter a positive number.")
    except ValueError:
        print("Invalid input. Please enter a number.")

print(f"\n✓ You will upload {num_files} file(s)")
print("─" * 80)

# Upload files one by one
all_dataframes = []
uploaded_filenames = []

for i in range(num_files):
    print(f"\n📁 Upload File {i+1} of {num_files}:")
    uploaded = files.upload()

    # Get the filename
    filename = list(uploaded.keys())[0]
    uploaded_filenames.append(filename)
    print(f"✓ File '{filename}' uploaded successfully!")

    # Read the Excel file (header is at row 19)
    temp_df = pd.read_excel(filename, skiprows=19)

    # Remove any rows that might be headers or empty
    temp_df = temp_df[temp_df['Symbol'].notna()].copy()
    temp_df = temp_df[temp_df['Symbol'] != 'Symbol'].copy()  # Remove if header row was included

    # Rename 'Profit' column to 'Actualised Profit'
    temp_df = temp_df.rename(columns={'Profit': 'Actualised Profit'})

    # Convert dates (with error handling)
    temp_df['Entry Date'] = pd.to_datetime(temp_df['Entry Date'], errors='coerce')
    temp_df['Exit Date'] = pd.to_datetime(temp_df['Exit Date'], errors='coerce')

    # Remove rows with invalid dates
    temp_df = temp_df[temp_df['Entry Date'].notna() & temp_df['Exit Date'].notna()].copy()

    # Convert numeric columns
    numeric_cols = ['Quantity', 'Buy Value', 'Sell Value', 'Actualised Profit']
    for col in numeric_cols:
        temp_df[col] = pd.to_numeric(temp_df[col], errors='coerce')

    # Remove any rows with NaN in critical columns
    temp_df = temp_df[temp_df['Quantity'].notna() & temp_df['Buy Value'].notna() & temp_df['Sell Value'].notna()].copy()

    all_dataframes.append(temp_df)
    print(f"   - Contains {len(temp_df)} trades")

# Combine all dataframes
print("\n" + "─" * 80)
print("COMBINING FILES...")
print("─" * 80)

df = pd.concat(all_dataframes, ignore_index=True)

print(f"\n✓ All {num_files} file(s) combined successfully!")
print(f"\nFiles uploaded:")
for idx, fname in enumerate(uploaded_filenames, 1):
    print(f"   {idx}. {fname}")

# Remove duplicates if any (based on key columns)
#initial_count = len(df)
#df = df.drop_duplicates(subset=['Symbol', 'Entry Date', 'Exit Date', 'Quantity', 'Buy Value', 'Sell Value'])
#duplicates_removed = initial_count - len(df)

#if duplicates_removed > 0:
 #   print(f"\n⚠️  Removed {duplicates_removed} duplicate trade(s)")

# Reset index
#df = df.reset_index(drop=True)

print("\n" + "=" * 80)
print("COMBINED DATA SUMMARY")
print("=" * 80)
print(f"\nTotal trades: {len(df)}")
print(f"Profitable trades: {len(df[df['Actualised Profit'] > 0])}")
print(f"Loss-making trades: {len(df[df['Actualised Profit'] < 0])}")
print(f"Unique stocks: {df['Symbol'].nunique()}")

# Show date range
print(f"\nDate range: {df['Entry Date'].min().strftime('%Y-%m-%d')} to {df['Exit Date'].max().strftime('%Y-%m-%d')}")

# Show stock list
print(f"\nTop 10 most traded stocks:")
stock_counts = df['Symbol'].value_counts()
for stock, count in stock_counts.head(10).items():
    print(f"   {stock}: {count} trades")
if len(stock_counts) > 10:
    print(f"   ... and {len(stock_counts) - 10} more stocks")

print("\n" + "─" * 80)
print("First few rows of combined data:")
print("─" * 80)
df[['Symbol', 'Entry Date', 'Exit Date', 'Quantity', 'Buy Value', 'Sell Value', 'Actualised Profit']].head(10)


TAX P&L FILE UPLOAD

How many Tax P&L files do you want to upload? (Enter a number): 4

✓ You will upload 4 file(s)
────────────────────────────────────────────────────────────────────────────────

📁 Upload File 1 of 4:


Saving taxpnl-SSW336-2024_2025-Q1-Q4 3.xlsx to taxpnl-SSW336-2024_2025-Q1-Q4 3.xlsx
✓ File 'taxpnl-SSW336-2024_2025-Q1-Q4 3.xlsx' uploaded successfully!
   - Contains 1867 trades

📁 Upload File 2 of 4:


Saving taxpnl-LWL704-2025_2026-Q1-Q3.xlsx to taxpnl-LWL704-2025_2026-Q1-Q3.xlsx
✓ File 'taxpnl-LWL704-2025_2026-Q1-Q3.xlsx' uploaded successfully!
   - Contains 284 trades

📁 Upload File 3 of 4:


Saving taxpnl-SSW336-2025_2026-Q1-Q3 2 (1).xlsx to taxpnl-SSW336-2025_2026-Q1-Q3 2 (1).xlsx
✓ File 'taxpnl-SSW336-2025_2026-Q1-Q3 2 (1).xlsx' uploaded successfully!
   - Contains 754 trades

📁 Upload File 4 of 4:


Saving taxpnl-LWL704-2024_2025-Q1-Q4-3.xlsx to taxpnl-LWL704-2024_2025-Q1-Q4-3.xlsx
✓ File 'taxpnl-LWL704-2024_2025-Q1-Q4-3.xlsx' uploaded successfully!
   - Contains 777 trades

────────────────────────────────────────────────────────────────────────────────
COMBINING FILES...
────────────────────────────────────────────────────────────────────────────────

✓ All 4 file(s) combined successfully!

Files uploaded:
   1. taxpnl-SSW336-2024_2025-Q1-Q4 3.xlsx
   2. taxpnl-LWL704-2025_2026-Q1-Q3.xlsx
   3. taxpnl-SSW336-2025_2026-Q1-Q3 2 (1).xlsx
   4. taxpnl-LWL704-2024_2025-Q1-Q4-3.xlsx

COMBINED DATA SUMMARY

Total trades: 3682
Profitable trades: 2199
Loss-making trades: 1481
Unique stocks: 171

Date range: 2023-08-17 to 2025-10-24

Top 10 most traded stocks:
   ANANTRAJ: 216 trades
   ELECON: 197 trades
   CDSL: 136 trades
   GOLDETF: 127 trades
   GOLDBEES: 115 trades
   ZOMATO: 96 trades
   ABB: 96 trades
   INDHOTEL: 93 trades
   KIMS: 80 trades
   CANBK: 73 trades
   ... and 161 mor

,Symbol,Entry Date,Exit Date,Quantity,Buy Value,Sell Value,Actualised Profit
0,JSFB,2024-04-30,2024-04-30,8,4534.00,4772.40,238.40
1,JSFB,2024-04-30,2024-04-30,1,566.75,596.55,29.80
2,JSFB,2024-04-30,2024-04-30,15,8501.25,8948.25,447.00
3,JSFB,2024-04-30,2024-04-30,15,8502.00,8948.25,446.25
4,JSFB,2024-04-30,2024-04-30,8,4534.40,4772.40,238.00
5,JSFB,2024-04-30,2024-04-30,11,6235.35,6562.05,326.70
6,JSFB,2024-04-30,2024-04-30,4,2267.60,2386.20,118.60
7,JSFB,2024-04-30,2024-04-30,25,14172.50,14913.75,741.25
8,JSFB,2024-04-30,2024-04-30,4,2267.60,2386.20,118.60
9,JSFB,2024-04-30,2024-04-30,1,566.90,596.55,29.65


## Step 3: Import Required Functions

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


In [4]:
# Mapping of incorrect → correct ticker symbols
rename_map = {
    "ZOMATO": "ETERNAL",          # Eternal Ltd (formerly Zomato)
    "DHABRIYA": "DHABRIYA",       # Dhabriya Polywood Ltd
    "TRIL": "TARIL",              # Transformers & Rectifiers (India) Ltd
    "SHISHIND": "SHISHIND",       # Shish Industries Ltd
    "BAJAJ-AUTO": "BAJAJ-AUTO",   # Bajaj Auto Ltd
    "BENARAS": "BNRS",            # Benares Hotels Ltd
    "KPT": "KPTI",                # KPT Industries Ltd
    "GUJTLRM": "GUJTLRM",         # Gujarat Toolroom Ltd
    "FLUIDOM": "FLUIDOM",         # Fluidomat Ltd
    "HBLPOWER": "HBLENGINE",      # HBL Power Systems Ltd (now HBL Engineering)
    "IIFLSEC": "IIFLSEC",         # IIFL Securities Ltd
    "CEINSYSTECH": "CEINSYSTECH", # Ceinsys Tech Ltd
    "SHILCTECH": "SHILCTECH"      # Shilchar Technologies Ltd
}

# Apply renaming to your DataFrame
df["Symbol"] = df["Symbol"].replace(rename_map)

# Optional: check which rows were updated
updated_rows = df[df["Symbol"].isin(rename_map.values())]
print("✅ Renaming complete. Updated tickers:")
print(updated_rows["Symbol"].unique())


✅ Renaming complete. Updated tickers:
['DHABRIYA' 'ETERNAL' 'TARIL' 'SHISHIND' 'BAJAJ-AUTO' 'BNRS' 'KPTI'
 'GUJTLRM' 'FLUIDOM' 'HBLENGINE' 'IIFLSEC' 'CEINSYSTECH' 'SHILCTECH']


## Step 4: Define Analysis Functions

In [5]:
import yfinance as yf

# 1️⃣ Define your BSE-only tickers
bse_only = [
    "DHABRIYA",
    "SHISHIND",
    "BNRS",
    "GUJTLRM",
    "FLUIDOM",
    "CEINSYSTECH",
    "SHILCTECH"
]

def download_stock_data(symbol, period='5y'):
    """
    Download stock data from Yahoo Finance.
    Uses .NS by default, .BO for known BSE-only tickers.

    Args:
        symbol: Stock symbol (without suffix)
        period: Time period for historical data
    Returns:
        DataFrame with historical data or None if not found
    """
    try:
        # Remove -E suffix if present (for ETFs etc.)
        clean_symbol = symbol.split('-')[0]

        # Use .BO for BSE-only, otherwise .NS
        exchange = ".BO" if clean_symbol in bse_only else ".NS"
        ticker = f"{clean_symbol}{exchange}"

        stock = yf.Ticker(ticker)
        data = stock.history(period=period)

        if data.empty:
            print(f"⚠️ No data found for {symbol} ({exchange.replace('.', '')})")
            return None

        print(f"✅ Downloaded {symbol} from {exchange.replace('.', '')}")
        return data

    except Exception as e:
        print(f"❌ Error downloading {symbol}: {str(e)}")
        return None


def calculate_moving_averages(data):
    """Calculate 50-day and 200-day moving averages"""
    if data is None or data.empty:
        return None

    data['50_DMA'] = data['Close'].rolling(window=50).mean()
    data['200_DMA'] = data['Close'].rolling(window=200).mean()

    return data

def get_price_vs_dma(symbol, date, stock_data):
    """Get price difference vs DMAs on a specific date"""
    target_date = pd.to_datetime(date).date()

    if symbol not in stock_data or stock_data[symbol] is None:
        return "No data available"

    df = stock_data[symbol]
    available_dates = df.index.date
    closest_date = min(available_dates, key=lambda x: abs((x - target_date).days))

    if abs((closest_date - target_date).days) > 5:
        return "Date too far from available data"

    try:
        day_data = df[df.index.date == closest_date].iloc[0]
        price = day_data['Close']
        dma_50 = day_data['50_DMA']
        dma_200 = day_data['200_DMA']

        if pd.isna(dma_50) or pd.isna(dma_200):
            return "DMA not available (insufficient history)"

        diff_50 = ((price - dma_50) / dma_50) * 100
        diff_200 = ((price - dma_200) / dma_200) * 100

        return f"50DMA: {diff_50:+.2f}%, 200DMA: {diff_200:+.2f}%"

    except Exception as e:
        return f"Error: {str(e)}"

def get_current_price_and_analysis(row, stock_data):
    """Get current price and calculate profit/loss vs sell date"""
    symbol = row['Symbol']
    sell_price = row['Sell Value'] / row['Quantity']  # Calculate per-unit sell price
    quantity = row['Quantity']

    if symbol not in stock_data or stock_data[symbol] is None:
        return None, None, "No data", None

    df = stock_data[symbol]

    try:
        current_price = df['Close'].iloc[-1]
        price_diff_pct = ((current_price - sell_price) / sell_price) * 100
        verdict = "Good call" if price_diff_pct < 0 else "Wrong early sell"
        profit_loss = (current_price - sell_price) * quantity

        return current_price, price_diff_pct, verdict, profit_loss

    except Exception as e:
        print(f"Error analyzing {symbol}: {str(e)}")
        return None, None, "Error", None

print("✓ Functions defined successfully!")
def get_dma_details(symbol, date, stock_data):
    """Get detailed DMA information including absolute values and differences"""
    target_date = pd.to_datetime(date).date()

    if symbol not in stock_data or stock_data[symbol] is None:
        return None, None, None, None, None

    df = stock_data[symbol]
    available_dates = df.index.date
    closest_date = min(available_dates, key=lambda x: abs((x - target_date).days))

    if abs((closest_date - target_date).days) > 5:
        return None, None, None, None, None

    try:
        day_data = df[df.index.date == closest_date].iloc[0]
        price = day_data['Close']
        dma_50 = day_data['50_DMA']
        dma_200 = day_data['200_DMA']

        if pd.isna(dma_50) or pd.isna(dma_200):
            return None, None, None, None, None

        # Calculate percentage difference from price to each DMA
        diff_50_pct = ((price - dma_50) / dma_50) * 100
        diff_200_pct = ((price - dma_200) / dma_200) * 100

        # Calculate signed percentage difference: (50_DMA - 200_DMA) / 200_DMA * 100
        # Positive = 50 DMA above 200 DMA (bullish)
        # Negative = 50 DMA below 200 DMA (bearish)
        dma_diff_pct = ((dma_50 - dma_200) / dma_200) * 100

        return diff_50_pct, diff_200_pct, dma_50, dma_200, dma_diff_pct

    except Exception as e:
        return None, None, None, None, None


✓ Functions defined successfully!


## Step 5: Download Historical Data for All Stocks

This will take a few minutes depending on the number of unique stocks...

In [6]:
# Get unique stock symbols
unique_stocks = df['Symbol'].unique()

print(f"Downloading data for {len(unique_stocks)} unique stocks...\n")

# Download and process data for each stock
stock_data = {}
for idx, symbol in enumerate(unique_stocks, 1):
    print(f"[{idx}/{len(unique_stocks)}] Downloading {symbol}...", end=' ')
    data = download_stock_data(symbol)

    if data is not None:
        data = calculate_moving_averages(data)
        stock_data[symbol] = data
        print(f"✓ ({len(data)} days)")
    else:
        print("✗ (Failed)")
        stock_data[symbol] = None

print(f"\n✓ Downloaded data for {sum(1 for v in stock_data.values() if v is not None)}/{len(unique_stocks)} stocks")


[1/169] Downloading JSFB... ✅ Downloaded JSFB from NS
✓ (420 days)
[2/169] Downloading EASEMYTRIP... ✅ Downloaded EASEMYTRIP from NS
✓ (1138 days)
[3/169] Downloading SELAN... ✅ Downloaded SELAN from NS
✓ (1236 days)
[4/169] Downloading ITDCEM... ✅ Downloaded ITDCEM from NS
✓ (1236 days)
[5/169] Downloading NETWEB... ✅ Downloaded NETWEB from NS
✓ (555 days)
[6/169] Downloading AVL... ✅ Downloaded AVL from NS
✓ (241 days)
[7/169] Downloading JINDALSAW... ✅ Downloaded JINDALSAW from NS
✓ (1237 days)
[8/169] Downloading MONARCH... ✅ Downloaded MONARCH from NS
✓ (940 days)
[9/169] Downloading TATAMOTORS... ✅ Downloaded TATAMOTORS from NS
✓ (1236 days)
[10/169] Downloading NCC... ✅ Downloaded NCC from NS
✓ (1237 days)
[11/169] Downloading ABB... ✅ Downloaded ABB from NS
✓ (1237 days)
[12/169] Downloading KAYNES... ✅ Downloaded KAYNES from NS
✓ (724 days)
[13/169] Downloading PRECWIRE... ✅ Downloaded PRECWIRE from NS
✓ (1237 days)
[14/169] Downloading TALBROAUTO... ✅ Downloaded TALBROAUTO f

ERROR:yfinance:$ATULAUTO.NS: possibly delisted; no price data found  (period=5y)


⚠️ No data found for ATULAUTO (NS)
✗ (Failed)
[23/169] Downloading ELECON... ✅ Downloaded ELECON from NS
✓ (1237 days)
[24/169] Downloading APLAPOLLO... ✅ Downloaded APLAPOLLO from NS
✓ (1237 days)
[25/169] Downloading TATAPOWER... ✅ Downloaded TATAPOWER from NS
✓ (1237 days)
[26/169] Downloading METROBRAND... ✅ Downloaded METROBRAND from NS
✓ (951 days)
[27/169] Downloading DHABRIYA... ✅ Downloaded DHABRIYA from BO
✓ (206 days)
[28/169] Downloading MARUTI... ✅ Downloaded MARUTI from NS
✓ (1237 days)
[29/169] Downloading M&M... ✅ Downloaded M&M from NS
✓ (1237 days)
[30/169] Downloading REPRO... ✅ Downloaded REPRO from NS
✓ (1237 days)
[31/169] Downloading TITAN... ✅ Downloaded TITAN from NS
✓ (1237 days)
[32/169] Downloading CONTROLPR... ✅ Downloaded CONTROLPR from NS
✓ (1237 days)
[33/169] Downloading CANBK... ✅ Downloaded CANBK from NS
✓ (1237 days)
[34/169] Downloading VBL... ✅ Downloaded VBL from NS
✓ (1237 days)
[35/169] Downloading INDHOTEL... ✅ Downloaded INDHOTEL from NS
✓ (12

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: BAJAJ.NS"}}}
ERROR:yfinance:$BAJAJ.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


⚠️ No data found for BAJAJ-AUTO (NS)
✗ (Failed)
[64/169] Downloading SBIN... 

...

✅ Downloaded SBIN from NS
✓ (1237 days)
[65/169] Downloading BNRS... ⚠️ No data found for BNRS (BO)
✗ (Failed)
[66/169] Downloading KPIGREEN... ✅ Downloaded KPIGREEN from NS
✓ (1052 days)
[67/169] Downloading KOTAKBANK... ✅ Downloaded KOTAKBANK from NS
✓ (1237 days)
[68/169] Downloading JAIBALAJI... ✅ Downloaded JAIBALAJI from NS
✓ (1237 days)
[69/169] Downloading MOTILALOFS... ✅ Downloaded MOTILALOFS from NS
✓ (1237 days)
[70/169] Downloading POONAWALLA... ✅ Downloaded POONAWALLA from NS
✓ (1237 days)
[71/169] Downloading KPITTECH... ✅ Downloaded KPITTECH from NS
✓ (1237 days)
[72/169] Downloading DBSTOCKBRO... ✅ Downloaded DBSTOCKBRO from NS
✓ (1237 days)
[73/169] Downloading LT... ✅ Downloaded LT from NS
✓ (1237 days)
[74/169] Downloading KPTI... 

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: KPTI.NS"}}}
ERROR:yfinance:$KPTI.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


⚠️ No data found for KPTI (NS)
✗ (Failed)
[75/169] Downloading GUJTLRM... ✅ Downloaded GUJTLRM from BO
✓ (895 days)
[76/169] Downloading PIIND... ✅ Downloaded PIIND from NS
✓ (1237 days)
[77/169] Downloading TITAGARH... ✅ Downloaded TITAGARH from NS
✓ (1237 days)
[78/169] Downloading PRICOLLTD... ✅ Downloaded PRICOLLTD from NS
✓ (1237 days)
[79/169] Downloading PHOENIXLTD... ✅ Downloaded PHOENIXLTD from NS
✓ (1237 days)
[80/169] Downloading PREMIERPOL... ✅ Downloaded PREMIERPOL from NS
✓ (1237 days)
[81/169] Downloading SUNDARMHLD... ✅ Downloaded SUNDARMHLD from NS
✓ (1236 days)
[82/169] Downloading ACE... ✅ Downloaded ACE from NS
✓ (1237 days)
[83/169] Downloading BIKAJI... ✅ Downloaded BIKAJI from NS
✓ (728 days)
[84/169] Downloading ITC... ✅ Downloaded ITC from NS
✓ (1237 days)
[85/169] Downloading HAL... ✅ Downloaded HAL from NS
✓ (1237 days)
[86/169] Downloading HDFCAMC... ✅ Downloaded HDFCAMC from NS
✓ (1237 days)
[87/169] Downloading SHAKTIPUMP... ✅ Downloaded SHAKTIPUMP from NS

ERROR:yfinance:$IIFLSEC.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


⚠️ No data found for IIFLSEC (NS)
✗ (Failed)
[101/169] Downloading WAAREEENER... ✅ Downloaded WAAREEENER from NS
✓ (248 days)
[102/169] Downloading DIXON... ✅ Downloaded DIXON from NS
✓ (1237 days)
[103/169] Downloading CEINSYSTECH... ✅ Downloaded CEINSYSTECH from BO
✓ (206 days)
[104/169] Downloading ANUP... ✅ Downloaded ANUP from NS
✓ (1237 days)
[105/169] Downloading SHILCTECH... ✅ Downloaded SHILCTECH from BO
✓ (1236 days)
[106/169] Downloading BLS... ✅ Downloaded BLS from NS
✓ (1237 days)
[107/169] Downloading SWIGGY... ✅ Downloaded SWIGGY from NS
✓ (236 days)
[108/169] Downloading KFINTECH... ✅ Downloaded KFINTECH from NS
✓ (697 days)
[109/169] Downloading KIMS... ✅ Downloaded KIMS from NS
✓ (1072 days)
[110/169] Downloading CAMS... ✅ Downloaded CAMS from NS
✓ (1237 days)
[111/169] Downloading DOMS... ✅ Downloaded DOMS from NS
✓ (457 days)
[112/169] Downloading ABSLAMC... ✅ Downloaded ABSLAMC from NS
✓ (1000 days)
[113/169] Downloading ICICIBANK... ✅ Downloaded ICICIBANK from NS


## Step 6: Analyze All Trades

Analyzing buy conditions, sell conditions, and verdicts...

In [7]:
print("\nAnalyzing trades...\n")
print("─" * 80)

# Create result dataframe
result_df = df[['Symbol', 'Entry Date', 'Exit Date', 'Quantity', 'Buy Value', 'Sell Value', 'Actualised Profit']].copy()

# Calculate per-unit prices for reference (these are intermediate calculations)
result_df['Buy Price Per Unit'] = result_df['Buy Value'] / result_df['Quantity']
result_df['Sell Price Per Unit'] = result_df['Sell Value'] / result_df['Quantity']

# Get detailed DMA information for entry date
print("Analyzing entry conditions (Buy)...")
entry_dma_details = result_df.apply(
    lambda row: get_dma_details(row['Symbol'], row['Entry Date'], stock_data),
    axis=1
)

# Split into separate columns
result_df['Buy Condition 50 DMA'] = entry_dma_details.apply(
    lambda x: f"{x[0]:+.2f}%" if x[0] is not None else "N/A"
)
result_df['Buy Condition 200 DMA'] = entry_dma_details.apply(
    lambda x: f"{x[1]:+.2f}%" if x[1] is not None else "N/A"
)
result_df['Entry 50 DMA Value'] = entry_dma_details.apply(
    lambda x: f"{x[2]:.2f}" if x[2] is not None else "N/A"
)
result_df['Entry 200 DMA Value'] = entry_dma_details.apply(
    lambda x: f"{x[3]:.2f}" if x[3] is not None else "N/A"
)
result_df['DMA Diff on Entry'] = entry_dma_details.apply(
    lambda x: f"{x[4]:+.2f}%" if x[4] is not None else "N/A"
)

# Get detailed DMA information for exit date
print("Analyzing exit conditions (Sell)...")
exit_dma_details = result_df.apply(
    lambda row: get_dma_details(row['Symbol'], row['Exit Date'], stock_data),
    axis=1
)

# Split into separate columns
result_df['Sell Condition 50 DMA'] = exit_dma_details.apply(
    lambda x: f"{x[0]:+.2f}%" if x[0] is not None else "N/A"
)
result_df['Sell Condition 200 DMA'] = exit_dma_details.apply(
    lambda x: f"{x[1]:+.2f}%" if x[1] is not None else "N/A"
)
result_df['Exit 50 DMA Value'] = exit_dma_details.apply(
    lambda x: f"{x[2]:.2f}" if x[2] is not None else "N/A"
)
result_df['Exit 200 DMA Value'] = exit_dma_details.apply(
    lambda x: f"{x[3]:.2f}" if x[3] is not None else "N/A"
)
result_df['DMA Diff on Exit'] = exit_dma_details.apply(
    lambda x: f"{x[4]:+.2f}%" if x[4] is not None else "N/A"
)

# Add Buy Verdict based on Actualised Profit
result_df['Buy Verdict'] = result_df['Actualised Profit'].apply(
    lambda x: "Good call" if x > 0 else "Bad call" if x < 0 else "Break-even"
)

# Add Current Analysis (for sell decisions)
print("Analyzing current prices vs sell prices...")
analysis_results = result_df.apply(
    lambda row: get_current_price_and_analysis(row, stock_data),
    axis=1
)

result_df['Current Price'] = analysis_results.apply(lambda x: x[0])
result_df['Price Change Since Sell'] = analysis_results.apply(lambda x: f"{x[1]:.2f}%" if x[1] is not None else "N/A")
result_df['Sell Verdict'] = analysis_results.apply(lambda x: x[2])
result_df['Opportunity Cost/Gain'] = analysis_results.apply(lambda x: f"{x[3]:,.2f}" if x[3] is not None else "N/A")

# Format Actualised Profit for display
result_df['Actualised Profit (Formatted)'] = result_df['Actualised Profit'].apply(lambda x: f"{x:,.2f}")

# Reorder columns for better readability
final_columns = [
    'Symbol',
    'Entry Date',
    'Buy Value',
    'Buy Price Per Unit',
    'Buy Condition 50 DMA',
    'Buy Condition 200 DMA',
    'Entry 50 DMA Value',
    'Entry 200 DMA Value',
    'DMA Diff on Entry',
    'Exit Date',
    'Sell Value',
    'Sell Price Per Unit',
    'Sell Condition 50 DMA',
    'Sell Condition 200 DMA',
    'Exit 50 DMA Value',
    'Exit 200 DMA Value',
    'DMA Diff on Exit',
    'Quantity',
    'Actualised Profit (Formatted)',
    'Buy Verdict',
    'Current Price',
    'Price Change Since Sell',
    'Sell Verdict',
    'Opportunity Cost/Gain'
]

result_df_display = result_df[final_columns].copy()

print("\n✓ Analysis complete!")
print("─" * 80)
print("\n" + "=" * 80)
print("ANALYSIS RESULTS")
print("=" * 80)

# Show summary statistics
good_buys = len(result_df[result_df['Buy Verdict'] == 'Good call'])
bad_buys = len(result_df[result_df['Buy Verdict'] == 'Bad call'])
good_sells = len(result_df[result_df['Sell Verdict'] == 'Good call'])
wrong_sells = len(result_df[result_df['Sell Verdict'] == 'Wrong early sell'])

print(f"\n📊 SUMMARY:")
print(f"   Buy Decisions: {good_buys} profitable, {bad_buys} loss-making")
print(f"   Sell Timing: {good_sells} good calls (price fell), {wrong_sells} early sells (price rose)")
print(f"   Success Rate: {(good_buys/len(result_df)*100):.1f}% profitable trades")

print("\n" + "─" * 80)
print("Detailed Results:")
print("─" * 80)

result_df_display



Analyzing trades...

────────────────────────────────────────────────────────────────────────────────
Analyzing entry conditions (Buy)...
Analyzing exit conditions (Sell)...
Analyzing current prices vs sell prices...

✓ Analysis complete!
────────────────────────────────────────────────────────────────────────────────

ANALYSIS RESULTS

📊 SUMMARY:
   Buy Decisions: 2199 profitable, 1481 loss-making
   Sell Timing: 1819 good calls (price fell), 1836 early sells (price rose)
   Success Rate: 59.7% profitable trades

────────────────────────────────────────────────────────────────────────────────
Detailed Results:
────────────────────────────────────────────────────────────────────────────────


,Symbol,Entry Date,Buy Value,Buy Price Per Unit,Buy Condition 50 DMA,Buy Condition 200 DMA,Entry 50 DMA Value,Entry 200 DMA Value,DMA Diff on Entry,Exit Date,...,Exit 50 DMA Value,Exit 200 DMA Value,DMA Diff on Exit,Quantity,Actualised Profit (Formatted),Buy Verdict,Current Price,Price Change Since Sell,Sell Verdict,Opportunity Cost/Gain
0,JSFB,2024-04-30,4534.00,566.75,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,8,238.40,Good call,453.600006,-23.96%,Good call,"-1,143.60"
1,JSFB,2024-04-30,566.75,566.75,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,1,29.80,Good call,453.600006,-23.96%,Good call,-142.95
2,JSFB,2024-04-30,8501.25,566.75,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,15,447.00,Good call,453.600006,-23.96%,Good call,"-2,144.25"
3,JSFB,2024-04-30,8502.00,566.80,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,15,446.25,Good call,453.600006,-23.96%,Good call,"-2,144.25"
4,JSFB,2024-04-30,4534.40,566.80,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,8,238.00,Good call,453.600006,-23.96%,Good call,"-1,143.60"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,GOLDETF,2025-01-13,1237.44,77.34,+2.49%,+6.94%,75.66,72.51,+4.34%,2025-03-24,...,82.91,75.61,+9.66%,16,135.84,Good call,117.839996,37.29%,Wrong early sell,512.16
3678,GOLDETF,2025-01-13,15158.64,77.34,+2.49%,+6.94%,75.66,72.51,+4.34%,2025-03-24,...,82.91,75.61,+9.66%,196,"1,664.04",Good call,117.839996,37.29%,Wrong early sell,"6,273.96"
3679,GOLDETF,2025-01-13,240063.36,77.34,+2.49%,+6.94%,75.66,72.51,+4.34%,2025-03-24,...,82.91,75.61,+9.66%,3104,"26,321.92",Good call,117.839996,37.31%,Wrong early sell,"99,390.07"
3680,GOLDETF,2025-02-10,581677.60,84.35,+9.34%,+15.08%,77.60,73.73,+5.25%,2025-03-24,...,82.91,75.61,+9.66%,6896,"10,137.12",Good call,117.839996,37.31%,Wrong early sell,"220,809.89"


## Step 7: View Results

In [8]:
print("\n" + "=" * 80)
print("ANALYSIS RESULTS")
print("=" * 80)

# Show summary statistics
good_buys = len(result_df[result_df['Buy Verdict'] == 'Good call'])
bad_buys = len(result_df[result_df['Buy Verdict'] == 'Bad call'])
good_sells = len(result_df[result_df['Sell Verdict'] == 'Good call'])
wrong_sells = len(result_df[result_df['Sell Verdict'] == 'Wrong early sell'])

print(f"\n📊 SUMMARY:")
print(f"   Buy Decisions: {good_buys} profitable, {bad_buys} loss-making")
print(f"   Sell Timing: {good_sells} good calls (price fell), {wrong_sells} early sells (price rose)")
print(f"   Success Rate: {(good_buys/len(result_df)*100):.1f}% profitable trades")

print("\n" + "─" * 80)
print("Detailed Results:")
print("─" * 80)

result_df_display


ANALYSIS RESULTS

📊 SUMMARY:
   Buy Decisions: 2199 profitable, 1481 loss-making
   Sell Timing: 1819 good calls (price fell), 1836 early sells (price rose)
   Success Rate: 59.7% profitable trades

────────────────────────────────────────────────────────────────────────────────
Detailed Results:
────────────────────────────────────────────────────────────────────────────────


,Symbol,Entry Date,Buy Value,Buy Price Per Unit,Buy Condition 50 DMA,Buy Condition 200 DMA,Entry 50 DMA Value,Entry 200 DMA Value,DMA Diff on Entry,Exit Date,...,Exit 50 DMA Value,Exit 200 DMA Value,DMA Diff on Exit,Quantity,Actualised Profit (Formatted),Buy Verdict,Current Price,Price Change Since Sell,Sell Verdict,Opportunity Cost/Gain
0,JSFB,2024-04-30,4534.00,566.75,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,8,238.40,Good call,453.600006,-23.96%,Good call,"-1,143.60"
1,JSFB,2024-04-30,566.75,566.75,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,1,29.80,Good call,453.600006,-23.96%,Good call,-142.95
2,JSFB,2024-04-30,8501.25,566.75,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,15,447.00,Good call,453.600006,-23.96%,Good call,"-2,144.25"
3,JSFB,2024-04-30,8502.00,566.80,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,15,446.25,Good call,453.600006,-23.96%,Good call,"-2,144.25"
4,JSFB,2024-04-30,4534.40,566.80,N/A,N/A,N/A,N/A,N/A,2024-04-30,...,N/A,N/A,N/A,8,238.00,Good call,453.600006,-23.96%,Good call,"-1,143.60"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,GOLDETF,2025-01-13,1237.44,77.34,+2.49%,+6.94%,75.66,72.51,+4.34%,2025-03-24,...,82.91,75.61,+9.66%,16,135.84,Good call,117.839996,37.29%,Wrong early sell,512.16
3678,GOLDETF,2025-01-13,15158.64,77.34,+2.49%,+6.94%,75.66,72.51,+4.34%,2025-03-24,...,82.91,75.61,+9.66%,196,"1,664.04",Good call,117.839996,37.29%,Wrong early sell,"6,273.96"
3679,GOLDETF,2025-01-13,240063.36,77.34,+2.49%,+6.94%,75.66,72.51,+4.34%,2025-03-24,...,82.91,75.61,+9.66%,3104,"26,321.92",Good call,117.839996,37.31%,Wrong early sell,"99,390.07"
3680,GOLDETF,2025-02-10,581677.60,84.35,+9.34%,+15.08%,77.60,73.73,+5.25%,2025-03-24,...,82.91,75.61,+9.66%,6896,"10,137.12",Good call,117.839996,37.31%,Wrong early sell,"220,809.89"


## Step 8: Detailed Summary Analytics

Breaking down your trading performance by financial year...

In [9]:
from datetime import datetime

def get_financial_year(date):
    date = pd.to_datetime(date)
    if date.month >= 4:
        return f"FY {date.year}-{str(date.year + 1)[-2:]}"
    else:
        return f"FY {date.year - 1}-{str(date.year)[-2:]}"

result_df['Entry FY'] = result_df['Entry Date'].apply(get_financial_year)
result_df['Exit FY'] = result_df['Exit Date'].apply(get_financial_year)

financial_years = sorted(result_df['Exit FY'].unique())

print("\n" + "=" * 80)
print("TRADING PERFORMANCE ANALYSIS - BY FINANCIAL YEAR")
print("=" * 80)

for fy in financial_years:
    fy_data = result_df[result_df['Exit FY'] == fy]

    print(f"\n{'─' * 80}")
    print(f"📅 {fy} ({len(fy_data)} trades)")
    print(f"{'─' * 80}")

    print("\n📊 Buy Decision Performance:")
    buy_verdict_counts = fy_data['Buy Verdict'].value_counts()
    for verdict, count in buy_verdict_counts.items():
        percentage = (count / len(fy_data)) * 100
        print(f"   {verdict}: {count} trades ({percentage:.1f}%)")

    fy_actual_profit = fy_data['Actualised Profit'].sum()

    print(f"\n💰 Total Actualised Profit/Loss:")
    print(f"   ₹{fy_actual_profit:,.2f}")
    if fy_actual_profit > 0:
        print(f"   Note: Net profit from all trades closed in {fy}")
    else:
        print(f"   Note: Net loss from all trades closed in {fy}")

    print("\n📊 Sell Timing Analysis:")
    sell_verdict_counts = fy_data['Sell Verdict'].value_counts()
    for verdict, count in sell_verdict_counts.items():
        if verdict not in ['No data', 'Error']:
            percentage = (count / len(fy_data)) * 100
            print(f"   {verdict}: {count} trades ({percentage:.1f}%)")

    fy_data_copy = fy_data.copy()
    fy_data_copy['Opp_numeric'] = fy_data_copy['Opportunity Cost/Gain'].apply(
        lambda x: float(x.replace(',', '')) if isinstance(x, str) and x != 'N/A' else 0
    )
    fy_opportunity = fy_data_copy['Opp_numeric'].sum()

    print(f"\n💸 Total Opportunity {'Cost' if fy_opportunity > 0 else 'Saved'}:")
    print(f"   ₹{abs(fy_opportunity):,.2f}")
    if fy_opportunity > 0:
        print(f"   Note: Additional profit missed by selling early in {fy}")
    else:
        print(f"   Note: Loss avoided by selling when you did in {fy}")

print(f"\n{'=' * 80}")
print(f"📈 ALL-TIME SUMMARY ({len(result_df)} total trades)")
print(f"{'=' * 80}")

print("\n📊 Overall Buy Decision Performance:")
buy_verdict_counts_all = result_df['Buy Verdict'].value_counts()
for verdict, count in buy_verdict_counts_all.items():
    percentage = (count / len(result_df)) * 100
    print(f"   {verdict}: {count} trades ({percentage:.1f}%)")

total_actual_profit = result_df['Actualised Profit'].sum()
print(f"\n💰 Total Actualised Profit/Loss (All-Time):")
print(f"   ₹{total_actual_profit:,.2f}")

print("\n📊 Overall Sell Timing Analysis:")
sell_verdict_counts_all = result_df['Sell Verdict'].value_counts()
for verdict, count in sell_verdict_counts_all.items():
    if verdict not in ['No data', 'Error']:
        percentage = (count / len(result_df)) * 100
        print(f"   {verdict}: {count} trades ({percentage:.1f}%)")

result_df_copy = result_df.copy()
result_df_copy['Opp_numeric'] = result_df_copy['Opportunity Cost/Gain'].apply(
    lambda x: float(x.replace(',', '')) if isinstance(x, str) and x != 'N/A' else 0
)
total_opportunity = result_df_copy['Opp_numeric'].sum()

print(f"\n💸 Total Opportunity {'Cost' if total_opportunity > 0 else 'Saved'} (All-Time):")
print(f"   ₹{abs(total_opportunity):,.2f}")

print("\n" + "─" * 80)
print("📌 Key Insights:")

fy_profit_summary = result_df.groupby('Exit FY')['Actualised Profit'].sum().sort_values()
best_profit_fy = fy_profit_summary.index[-1]
worst_profit_fy = fy_profit_summary.index[0]

print(f"   ✅ Best Profit FY: {best_profit_fy} (₹{fy_profit_summary[best_profit_fy]:,.2f})")
print(f"   ⚠️  Worst Profit FY: {worst_profit_fy} (₹{fy_profit_summary[worst_profit_fy]:,.2f})")

good_buy_calls = buy_verdict_counts_all.get('Good call', 0)
good_sell_calls = sell_verdict_counts_all.get('Good call', 0)
buy_success_rate = (good_buy_calls / len(result_df)) * 100
sell_success_rate = (good_sell_calls / len(result_df)) * 100

print(f"\n   🎯 Buy Success Rate: {buy_success_rate:.1f}% (Profitable trades)")
print(f"   🎯 Sell Timing Success Rate: {sell_success_rate:.1f}% (Avoided losses)")

avg_profit = result_df['Actualised Profit'].mean()
print(f"\n   📊 Average Profit per Trade: ₹{avg_profit:,.2f}")

print("\n" + "=" * 80)
print("✓ Analysis complete! Detailed results will be saved in the next step.")
print("=" * 80)


TRADING PERFORMANCE ANALYSIS - BY FINANCIAL YEAR

────────────────────────────────────────────────────────────────────────────────
📅 FY 2024-25 (2644 trades)
────────────────────────────────────────────────────────────────────────────────

📊 Buy Decision Performance:
   Good call: 1515 trades (57.3%)
   Bad call: 1127 trades (42.6%)
   Break-even: 2 trades (0.1%)

💰 Total Actualised Profit/Loss:
   ₹1,115,795.80
   Note: Net profit from all trades closed in FY 2024-25

📊 Sell Timing Analysis:
   Good call: 1452 trades (54.9%)
   Wrong early sell: 1165 trades (44.1%)

💸 Total Opportunity Cost:
   ₹80,218.63
   Note: Additional profit missed by selling early in FY 2024-25

────────────────────────────────────────────────────────────────────────────────
📅 FY 2025-26 (1038 trades)
────────────────────────────────────────────────────────────────────────────────

📊 Buy Decision Performance:
   Good call: 684 trades (65.9%)
   Bad call: 354 trades (34.1%)

💰 Total Actualised Profit/Loss:
   

## Step 9: Save and Download Results

In [10]:
# Save to CSV with ALL columns for validation
# Including original total values, per-unit prices, and all analysis columns
output_df = result_df[[
    # Core trading data (ORIGINAL VALUES - NOT MODIFIED)
    'Symbol',
    'Entry Date',
    'Exit Date',
    'Quantity',
    'Buy Value',              # Original total buy value
    'Sell Value',             # Original total sell value
    'Actualised Profit',      # Original profit/loss

    # Calculated per-unit prices (for reference)
    'Buy Price Per Unit',
    'Sell Price Per Unit',

    # Enhanced Buy/Entry columns
    'Buy Condition 50 DMA',
    'Buy Condition 200 DMA',
    'Entry 50 DMA Value',
    'Entry 200 DMA Value',
    'DMA Diff on Entry',

    # Enhanced Sell/Exit columns
    'Sell Condition 50 DMA',
    'Sell Condition 200 DMA',
    'Exit 50 DMA Value',
    'Exit 200 DMA Value',
    'DMA Diff on Exit',

    # Analysis columns
    'Buy Verdict',
    'Current Price',
    'Price Change Since Sell',
    'Sell Verdict',
    'Opportunity Cost/Gain',
]].copy()

# Add any other columns that exist in result_df but not in the list above
existing_columns = set(output_df.columns)
all_columns = set(result_df.columns)
missing_columns = all_columns - existing_columns

if missing_columns:
    print(f"\nℹ️  Adding {len(missing_columns)} additional columns found in result_df:")
    for col in sorted(missing_columns):
        print(f"   - {col}")
        output_df[col] = result_df[col]

output_filename = 'analyzed_trades_complete.csv'
output_df.to_csv(output_filename, index=False)
print(f"\n✓ Results saved to '{output_filename}'")
print(f"   Total columns in CSV: {len(output_df.columns)}")
print(f"   Total rows: {len(output_df)}")

# Display column summary
print(f"\n📋 Columns in exported CSV:")
for i, col in enumerate(output_df.columns, 1):
    print(f"   {i:2d}. {col}")

# Validation check - ensure original values are preserved
print(f"\n✅ Validation: Original columns preserved")
print(f"   - Buy Value sum: ₹{output_df['Buy Value'].sum():,.2f}")
print(f"   - Sell Value sum: ₹{output_df['Sell Value'].sum():,.2f}")
print(f"   - Actualised Profit sum: ₹{output_df['Actualised Profit'].sum():,.2f}")

# Download the file
print("\n" + "─" * 80)
print("Downloading the results file...")
files.download(output_filename)
print("\n✓ Download complete!")


#possible to fix stock spilt inaccuracies?

#possible to fix stock name changes and "-" issues

###########################################################

#understand what you are doing right or wrong individually
#is it influenced by news?
#understand if there is a pattern wrt dma
#share the selling points with llm and check once as well
#market just fell and you acted to protect capital?, acted on SL rules?



# TODO - FUTURE ENHANCEMENTS:
# [ ] Fix stock split inaccuracies (detect and adjust historical prices)
# [ ] Handle stock name changes and "-" issues (create mapping)
# [ ] Understand what you are doing right or wrong individually
# [ ] Check if decisions were influenced by news (integrate news API?)
# [ ] Find patterns with respect to DMA (statistical analysis)
# [ ] Share selling points with LLM for analysis
# [ ] Add context: Did market fall? Stop-loss triggered?
# [ ] Add sector/industry analysis
# [ ] Add holding period analysis
# [ ] Compare against index performance (Nifty/Sensex)

'''
Fix there
 [92/169] Downloading BAJAJ-AUTO... ERROR:yfinance:$BAJAJ.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$BNRS.BO: possibly delisted; no price data found  (period=5y)
⚠️ No data found for BAJAJ-AUTO (NS)
✗ (Failed)

[93/169] Downloading BNRS... ⚠️ No data found for BNRS (BO)
✗ (Failed)

[99/169] Downloading KPTI... ERROR:yfinance:$KPTI.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
⚠️ No data found for KPTI (NS)
✗ (Failed)

[118/169] Downloading IIFLSEC... ERROR:yfinance:$IIFLSEC.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
⚠️ No data found for IIFLSEC (NS)
✗ (Failed)
'''




ℹ️  Adding 3 additional columns found in result_df:
   - Actualised Profit (Formatted)
   - Entry FY
   - Exit FY

✓ Results saved to 'analyzed_trades_complete.csv'
   Total columns in CSV: 27
   Total rows: 3682

📋 Columns in exported CSV:
    1. Symbol
    2. Entry Date
    3. Exit Date
    4. Quantity
    5. Buy Value
    6. Sell Value
    7. Actualised Profit
    8. Buy Price Per Unit
    9. Sell Price Per Unit
   10. Buy Condition 50 DMA
   11. Buy Condition 200 DMA
   12. Entry 50 DMA Value
   13. Entry 200 DMA Value
   14. DMA Diff on Entry
   15. Sell Condition 50 DMA
   16. Sell Condition 200 DMA
   17. Exit 50 DMA Value
   18. Exit 200 DMA Value
   19. DMA Diff on Exit
   20. Buy Verdict
   21. Current Price
   22. Price Change Since Sell
   23. Sell Verdict
   24. Opportunity Cost/Gain
   25. Actualised Profit (Formatted)
   26. Entry FY
   27. Exit FY

✅ Validation: Original columns preserved
   - Buy Value sum: ₹32,789,871.58
   - Sell Value sum: ₹34,190,239.42
   - Actua

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ Download complete!


'\nFix there\n [92/169] Downloading BAJAJ-AUTO... ERROR:yfinance:$BAJAJ.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")\nERROR:yfinance:$BNRS.BO: possibly delisted; no price data found  (period=5y)\n⚠️ No data found for BAJAJ-AUTO (NS)\n✗ (Failed)\n\n[93/169] Downloading BNRS... ⚠️ No data found for BNRS (BO)\n✗ (Failed)\n\n[99/169] Downloading KPTI... ERROR:yfinance:$KPTI.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")\n⚠️ No data found for KPTI (NS)\n✗ (Failed)\n\n[118/169] Downloading IIFLSEC... ERROR:yfinance:$IIFLSEC.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")\n⚠️ No data found for IIFLSEC (NS)\n✗ (Failed)\n'

In [11]:


#possible to fix stock spilt inaccuracies?

#possible to fix stock name changes and "-" issues

###########################################################

#understand what you are doing right or wrong individually
#is it influenced by news?
#understand if there is a pattern wrt dma
#share the selling points with llm and check once as well
#market just fell and you acted to protect capital?, acted on SL rules?


In [12]:


# TODO - FUTURE ENHANCEMENTS:
# [ ] Fix stock split inaccuracies (detect and adjust historical prices)
# [ ] Handle stock name changes and "-" issues (create mapping)
# [ ] Understand what you are doing right or wrong individually
# [ ] Check if decisions were influenced by news (integrate news API?)
# [ ] Find patterns with respect to DMA (statistical analysis)
# [ ] Share selling points with LLM for analysis
# [ ] Add context: Did market fall? Stop-loss triggered?
# [ ] Add sector/industry analysis
# [ ] Add holding period analysis
# [ ] Compare against index performance (Nifty/Sensex)


In [13]:
'''
Fix there
 [92/169] Downloading BAJAJ-AUTO... ERROR:yfinance:$BAJAJ.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$BNRS.BO: possibly delisted; no price data found  (period=5y)
⚠️ No data found for BAJAJ-AUTO (NS)
✗ (Failed)

[93/169] Downloading BNRS... ⚠️ No data found for BNRS (BO)
✗ (Failed)

[99/169] Downloading KPTI... ERROR:yfinance:$KPTI.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
⚠️ No data found for KPTI (NS)
✗ (Failed)

[118/169] Downloading IIFLSEC... ERROR:yfinance:$IIFLSEC.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
⚠️ No data found for IIFLSEC (NS)
✗ (Failed)
'''


'\nFix there\n [92/169] Downloading BAJAJ-AUTO... ERROR:yfinance:$BAJAJ.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")\nERROR:yfinance:$BNRS.BO: possibly delisted; no price data found  (period=5y)\n⚠️ No data found for BAJAJ-AUTO (NS)\n✗ (Failed)\n\n[93/169] Downloading BNRS... ⚠️ No data found for BNRS (BO)\n✗ (Failed)\n\n[99/169] Downloading KPTI... ERROR:yfinance:$KPTI.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")\n⚠️ No data found for KPTI (NS)\n✗ (Failed)\n\n[118/169] Downloading IIFLSEC... ERROR:yfinance:$IIFLSEC.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")\n⚠️ No data found for IIFLSEC (NS)\n✗ (Failed)\n'

## 🎉 Analysis Complete!

### What to do next:

1. **Review Buy Decisions**: Look at "Buy Conditions" and "Buy Verdict" to learn from profitable vs loss-making entries
2. **Review Sell Timing**: Check "Sell Conditions" and "Sell Verdict" to understand if you sold at the right time
3. **Study Patterns**: Look for patterns in market conditions (DMA positions) when you made good vs bad decisions
4. **Calculate Impact**: Review "Actualised Profit" (your actual gain/loss) and "Opportunity Cost/Gain" (what you missed/saved)

### Understanding the output:

**Buy Conditions**: Shows where the price was relative to moving averages when you bought
- Positive % = Price was above DMA (expensive, trending up)
- Negative % = Price was below DMA (cheap, potential value)

**Buy Verdict**:
- "Good call" = You made profit (Actualised Profit > 0)
- "Bad call" = You took a loss (Actualised Profit < 0)

**Sell Conditions**: Shows where the price was relative to moving averages when you sold
- Positive % = Sold when price was above DMA (strength)
- Negative % = Sold when price was below DMA (weakness)

**Sell Verdict**:
- "Good call" = Price went down after you sold (avoided further loss)
- "Wrong early sell" = Price went up after you sold (missed gains)

**Opportunity Cost/Gain**:
- Positive = Money you could have made by holding longer
- Negative = Money you saved by selling when you did

---

**Pro Tip**: Keep this notebook and re-run it with updated tax P&L files to track your trading decisions over time!